<a href="https://colab.research.google.com/github/sjanorkar/LLM-finetuning/blob/main/finetune_clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install bitsandbytes==0.41.3
!pip3 install peft==0.11.1
!pip3 install trl==0.8.6
!pip3 install accelerate==0.30.1
!pip3 install datasets==2.19.2
!pip3 install transformers==4.41.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 25.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-many

In [2]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from peft import (LoraConfig, get_peft_model, get_peft_model_state_dict)
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [19]:
dataset = load_dataset("Vezora/Tested-22k-Python-Alpaca", split="train")
dataset

instruction = dataset["instruction"]
input = dataset["input"]
output = dataset["output"]

import datasets
from datasets import Dataset

temp_dataset_1 = Dataset.from_dict({"instruction": instruction[0:1000], "input": input[0:1000], "output": output[0:1000]})
dataset_1 = datasets.DatasetDict({"train": temp_dataset_1})
dataset_1

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1000
    })
})

In [4]:
dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 22608
})

In [20]:
train_val = dataset_1["train"].train_test_split(
    test_size=100, shuffle=True, seed=42
)
train_data = (
    train_val["train"]#.map(generate_and_tokenize_prompt)
)
val_data = (
    train_val["test"]#.map(generate_and_tokenize_prompt)
)

In [6]:
val_data

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 100
})

In [1]:
#model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
model_id = "tiiuae/falcon-rw-1b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto")
model.config.use_cache = False

NameError: name 'AutoTokenizer' is not defined

In [22]:
%%time
inputs = tokenizer('''Write a python code to Output the factorial of 7 using a recursive function and without using any loops.''', return_tensors="pt", return_attention_mask=False)


outputs = model.generate(**inputs, max_length=300)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using 

In [23]:
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT= 0.05
LORA_TARGET_MODULES = [
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
    "lm_head",
    "query_key_value"
]

BATCH_SIZE = 64
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 400
OUTPUT_DIR = "/home/finetuned/falcon-rw-1b/Tested-22k-Python-Alpaca"

In [24]:
peft_params = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
  )

In [25]:
model = get_peft_model(model, peft_params)
model.print_trainable_parameters()

trainable params: 1,675,264 || all params: 1,313,300,480 || trainable%: 0.1276


In [26]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=300,
    logging_steps=300,
    learning_rate=2e-4,
    evaluation_strategy="steps",
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [27]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['instruction'])):
        text = f"### Question: {example['instruction'][i]}\n ### Answer: {example['output'][i]}"
        output_texts.append(text)
    return output_texts

#print(formatting_prompts_func(train_data))

['### Question: Create a python program to count how many times a word appears in a given string, but exclude any occurrences of the word that are within parentheses or square brackets. Additionally, the program should ignore any punctuation marks and consider the word as case insensitive. The program should also remove any duplicate words in the string before counting. For example, in the given string "The quick (brown) fox jumps over the lazy [dog]", the word "Brown" should not be counted, even if it is capitalized or followed by punctuation marks. The program should return a sorted dictionary with the word count as values.\n ### Answer: Here\'s a Python program that fulfills the given requirements:\n\n```python\nimport re\nfrom collections import defaultdict\n\ndef count_word_occurrences(string):\n    # Remove punctuation marks\n    string = re.sub(r\'[^\\w\\s]\', \'\', string)\n    \n    # Remove duplicate words\n    words = set(string.lower().split())\n    \n    # Count word occur

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


response_template = "### Answer: "
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
MAX_SEQ_LEN = 3056

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    formatting_func=formatting_prompts_func,
    #dataset_text_field="text",
    data_collator=collator,
    max_seq_length=MAX_SEQ_LEN,
    args=training_params,
    packing=False,
)

# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_params,
#     dataset_text_field="text",
#     max_seq_length=None,
#     tokenizer=tokenizer,
#     args=training_params,

# )

trainer.train()

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:141: UserWarning: Could not find response key `### Answer: ` in the following instance: ### Question: Implement a custom data structure to store a list of strings that supports the following operations with the specified time complexities:

1. Insertion: Add a string to the data structure - O(log n) time complexity
2. Deletion: Remove a specific string from the data structure - O(log n) time complexity
3. Search: Find a specific string in the data structure - O(log n) time complexity

Additionally, design an algorithm to sort the list of strings in descending order of their lengths, while minimizing the space complexity. The sorting algorithm should have a time complexity of O(n log n), where n is the number of strings in the list. It should also be stable, meaning that strings with equal lengths should retain their original order.
 ### Answer: To implement a custom data structure that supports insertion, deletion, and search

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.46 GiB. GPU 

In [14]:
%%time
inputs = tokenizer('''Write a python code to Output the factorial of 7 using a recursive function and without using any loops.''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1659: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using a recursive function and without using any loops.
Write a python code to Output the factorial of 7 using 

In [15]:
%%time
inputs = tokenizer('''Write a python code to return True if number is even else return false''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=300)
text = tokenizer.batch_decode(outputs)[0]
print(text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write a python code to return True if number is even else return false
Write 